# Financial Model Backtesting: Are historic stock returns compatible with a  stochastic model specified by the first four moments?

This documents builds on the know-how and methodology of modelBacktesting_standardRiskFactor.ipynb in the same repo. This document implements a backtesting methodology to assess if a given stock price process
is compatible with a predefined stochastic model. The predefined stochastic model is characterized by prescribing the first four moments of each increments distribution.

The document is a beta version, currently under construction....

In [ ]:
# Let's load what we will need
import pandas as pd
import numpy as np
import statsmodels.sandbox.distributions.extras as extras
import statsmodels.distributions.empirical_distribution as empirical_distribution
import scipy.interpolate as interpolate
import scipy.stats as ss
import matplotlib.pyplot as plt  
import matplotlib.patches as mpatches

In [ ]:
#Load historic returns data

historicReturns = pd.read_csv('monthly_returns_forward.txt')

historicReturns['lead_ret'] = pd.to_numeric(historicReturns['lead_ret'], errors='coerce')
historicReturns['lead_retx'] = pd.to_numeric(historicReturns['lead_retx'], errors='coerce')
historicReturns=historicReturns.dropna()

In [ ]:
#Load the moments data. All moments are interpreted as desribing a distribution of returns one month into the future.

datesDf = pd.read_csv('all_dates.txt', header=None)
permnoDf = pd.read_csv('permno.txt', header=None)

expectedReturnsDf = pd.read_csv('P_ER.txt', header=None)
varianceDf = pd.read_csv('P_variance.txt', header=None)
skewnessDf = pd.read_csv('P_skewness.txt', header=None)
kurtosisDf = pd.read_csv('P_kurtosis.txt', header=None)

## This is running a PIT based test for a certain PERMNO

In [ ]:
permnoId = 0

referenceSampleSize = 10000

permno = permnoDf.values[permnoId,0]

print(permno)

In [ ]:
# slice historic returns appropriately and format time data

historicReturnsPermno = historicReturns[historicReturns['PERMNO']==permno][['DATE','lead_ret']]

In [ ]:
# useful dates

historicReturnsPermnoUsefulDates = []
for i in range(historicReturnsPermno['DATE'].shape[0]):
    historicReturnsPermnoUsefulDates.append( historicReturnsPermno['DATE'].values.astype('str')[i][4:6] + '/' + historicReturnsPermno['DATE'].values.astype('str')[i][6:] + '/' + historicReturnsPermno['DATE'].values.astype('str')[i][:4])

In [ ]:
#Create a reference sample from the model. This is interpreted as a sample of distribution outcomes.

def sampleFromDistributionWithSpecifiedMoments(mu, sigma, skew, kurt, size, sd_wide=10):
   f = extras.pdf_mvsk([mu, sigma, skew, kurt])
   x = np.linspace(mu - sd_wide * sigma, mu + sd_wide * sigma, num=500)
   y = [f(i) for i in x]
   yy = np.cumsum(y) / np.sum(y)
   inv_cdf = interpolate.interp1d(yy, x, fill_value="extrapolate")
   rr = np.random.rand(size)

   return inv_cdf(rr)

In [ ]:
sampledModelIncrements = np.empty((datesDf.shape[0], referenceSampleSize))


# loop over all dates and get the reference samples:
for dateId in range(datesDf.shape[0]):
    sampledModelIncrements[dateId,:] = sampleFromDistributionWithSpecifiedMoments(
        expectedReturnsDf.values[permnoId, dateId],
        varianceDf.values[permnoId, dateId],
        skewnessDf.values[permnoId, dateId],
        kurtosisDf.values[permnoId, dateId],
        referenceSampleSize)
    
# from all sampled model increments we subtract 1 as this is their current gauging (gross returns +1)
sampledModelIncrements -=1

In [ ]:
#Here are 20 typical paths of your model. Where NAN are given we extrapolate flat.

modelSamplePaths = np.concatenate([np.zeros((1,sampledModelIncrements.shape[1])),np.nancumsum(sampledModelIncrements,0)],0)

plt.figure(figsize=(20, 10),facecolor='yellow')
plt.plot(modelSamplePaths[:,0:20], c = 'grey', alpha = 0.3)
#plt.plot(goodPath.T, c = 'green')

plt.xlabel('Time')
plt.ylabel('Price')
plt.ylim([-0.5,4])

plt.show()

In [ ]:
# Start by getting the ECDFs at different time steps

ECDFs = {}
for dateId in range(datesDf.shape[0]):
    if np.isnan(sampledModelIncrements[dateId,:]).any():
        continue
    ECDFs[dateId] = empirical_distribution.ECDF(sampledModelIncrements[dateId,:])

In [ ]:
# Computing the PIT values of the permno under consideration

PITs = np.empty(max(ECDFs))
PITs[:] = np.nan
historicPath  = {}
historicPath[0] = 0.0
for key, ECDF in ECDFs.items():
    backtestingDate = datesDf[1][key]
    
    if not backtestingDate in historicReturnsPermnoUsefulDates:
        continue
    historicPath[key + 1] = historicReturnsPermno['lead_ret'].values[historicReturnsPermnoUsefulDates.index(backtestingDate)]
    if historicReturnsPermnoUsefulDates.index(backtestingDate) + 1 == historicReturnsPermno.shape[0]:
        continue
              
    realizedReturn = historicReturnsPermno['lead_ret'].values[historicReturnsPermnoUsefulDates.index(backtestingDate) + 1] - historicReturnsPermno['lead_ret'].values[historicReturnsPermnoUsefulDates.index(backtestingDate)]

        
    PITs[key] = ECDF(realizedReturn)

In [ ]:
#Let's compare 20 model paths versus the realized path of the model over the horizon
#Here are 20 typical paths of your model. Where NAN are given we extrapolate flat.

modelSamplePaths = np.concatenate([np.zeros((1,sampledModelIncrements.shape[1])),np.nancumsum(sampledModelIncrements,0)],0)

historicPathNP = np.empty(modelSamplePaths.shape[0])
historicPathNP[:] = np.nan
for key, entry in historicPath.items():
    historicPathNP[key] = entry

plt.figure(figsize=(20, 10),facecolor='yellow')
plt.plot(modelSamplePaths[:,0:20], c = 'grey', alpha = 0.3)
plt.plot(historicPathNP, '-*', c = 'green')

plt.xlabel('Time')
plt.ylabel('Price')
plt.ylim([-0.5,4])

plt.show()

In [ ]:
#Plot PIT values and histograms to visually assess if they are uniform

fig, axs = plt.subplots(2, 1, figsize=(20, 10))
axs[0].plot(PITs, 'o')
axs[1].hist(PITs, bins = 10)

In [ ]:
# We run classical statistical tests to check if the sample truely came from the model
validPITs = PITs[ np.logical_not(np.isnan(PITs))]

ksTest = ss.kstest(validPITs, 'uniform') #Kholmogorov Smirnoff Test
cmTest = ss.cramervonmises(validPITs, 'uniform') #Cramer von Mises Test


In [ ]:
#Print the measured test statistics and pValues.
print('Number of valid PITs ' + str(validPITs.shape[0]))
print('KS test: ' + str(ksTest))
print('CM test: ' + str(cmTest))

## Wrapped loop over multiple PERMNOs

In [ ]:
# Wrap data cleaning and computation of PITS into a minimal function

def minimalBacktest(permnoId, historicReturnsPermnoUsefulDates):
    
    sampledModelIncrements = np.empty((datesDf.shape[0], referenceSampleSize))

    for dateId in range(datesDf.shape[0]):
        sampledModelIncrements[dateId,:] = sampleFromDistributionWithSpecifiedMoments(
            expectedReturnsDf.values[permnoId, dateId],
            varianceDf.values[permnoId, dateId],
            skewnessDf.values[permnoId, dateId],
            kurtosisDf.values[permnoId, dateId],
            referenceSampleSize)

    sampledModelIncrements -=1
    
    ECDFs = {}
    for dateId in range(datesDf.shape[0]):
        if np.isnan(sampledModelIncrements[dateId,:]).any():
            continue
        ECDFs[dateId] = empirical_distribution.ECDF(sampledModelIncrements[dateId,:])

    PITs = np.empty(max(ECDFs))
    PITs[:] = np.nan

    for key, ECDF in ECDFs.items():
        backtestingDate = datesDf[1][key]
        if not backtestingDate in historicReturnsPermnoUsefulDates:
            continue
        if historicReturnsPermnoUsefulDates.index(backtestingDate) + 1 == historicReturnsPermno.shape[0]:
            continue

        realizedReturn = historicReturnsPermno['lead_ret'].values[historicReturnsPermnoUsefulDates.index(backtestingDate) + 1] - historicReturnsPermno['lead_ret'].values[historicReturnsPermnoUsefulDates.index(backtestingDate)]
        PITs[key] = ECDF(realizedReturn)
        
    validPITs = PITs[np.logical_not(np.isnan(PITs))]

    ksTest = ss.kstest(validPITs, 'uniform')
    cmTest = ss.cramervonmises(validPITs, 'uniform')

    return (validPITs.shape[0], ksTest, cmTest)

In [ ]:
testResults = {}

for permnoId in range(permnoDf.values.shape[0]):
    permno = permnoDf.values[permnoId,0]
    print('Running for PERMNO ' + str(permno))
    historicReturnsPermno = historicReturns[historicReturns['PERMNO']==permno][['DATE','lead_ret']]
    if not historicReturnsPermno.values.size:
        testResults[permno] = 'NoReturnsData'
        continue
        
    historicReturnsPermnoUsefulDates = []
    for i in range(historicReturnsPermno['DATE'].shape[0]):
        historicReturnsPermnoUsefulDates.append( historicReturnsPermno['DATE'].values.astype('str')[i][4:6] + '/' + historicReturnsPermno['DATE'].values.astype('str')[i][6:] + '/' + historicReturnsPermno['DATE'].values.astype('str')[i][:4])
    
    try:
        testResults[permno] = minimalBacktest(permnoId, historicReturnsPermnoUsefulDates)
    except:
        testResults[permno] = 'TestsFailed'
    


In [ ]:
print(testResults)